# Volatility Traces
> Walkthrough
+ categories: [Volatility, Memory Foresenics]
+ source: [CybersDefenderos](https://cyberdefenders.org/blueteam-ctf-challenges/volatility-traces/)

## Overview
As a digital forensics analyst at CyberResponse Inc., you are faced with a critical incident involving a sophisticated malware infiltration at a multinational corporation. The malware has covertly compromised critical systems and initiated unauthorized activities, posing a severe threat to the organization’s sensitive data and operational integrity.

Your task is to conduct a comprehensive analysis of a captured memory dump to trace the malware's hidden activities. The goal is to meticulously decode the sequence of events, identify the malware's functionalities, and understand its mechanisms of action. This analysis is pivotal in uncovering how the malware evaded detection and executed its malicious payload.

**Tools:**
+ Volatility3
+ strings, grep

**Process Timeline**   

---

```
4 (smss.exe)
└── 404 (wininit.exe)
    └── 488 (csrss.exe)
        └── 568 (winlogon.exe)
            └── 3764 (userinit.exe)
                └── 3800 (explorer.exe)
                    └── 4596 (InvoiceCheckList.exe)
                        ├── 7656 (powershell.exe   "C:\Windows\System32\WindowsPowerShell\v1.0\powershell.exe" Add-MpPreference -ExclusionPath "C:\Users\Lee\AppData\Roaming\HcdmIYYf.exe")
                        │   └── 7220 (conhost.exe)
                        ├── 2816 (RegSvcs.exe)
                        ├── 4164 (RegSvcs.exe)
                        ├── 6796 (RegSvcs.exe)
                        ├── 6448 (RegSvcs.exe)
                        ├── 6980 (powershell.exe   "C:\Windows\System32\WindowsPowerShell\v1.0\powershell.exe" Add-MpPreference -ExclusionPath "C:\Users\Lee\AppData\Local\Temp\InvoiceCheckList.exe"
                        │   └── 3940 (conhost.exe)
                        └── 3512 (schtasks.exe)
                            └── 6364 (conhost.exe)
```

---

**Q1:** Identifying the parent process reveals the source and potential additional malicious activity. What is the name of the suspicious process that spawned two malicious PowerShell processes?  
**A:** 'InvoiceCheckList.exe'  

> Our initial attempt was to locate the parent process using `python3 vol.py -f file.dmp windows.psscan.PsScan`, which worked as expected, however the process name was truncated, not giving us the full process name. According to this [source](https://github.com/volatilityfoundation/volatility/issues/734) the data structure used to track the process names in the kernel only has room for the first 16 characters of the process name.
>
> 4596	3800	**InvoiceCheckLi**	0xb882f107e080	0	-	1	False	2024-05-02 06:57:42.000000 	2024-05-02 06:58:00.000000 	Disabled

As a workaround I was able to gather the full filename by using the 'windows.cmdline.CmdLine' module.   

`vol.py -f memory.dmp windows.cmdline.CmdLine`   

6980	powershell.exe	"C:\Windows\System32\WindowsPowerShell\v1.0\powershell.exe" Add-MpPreference -ExclusionPath **C:\Users\Lee\AppData\Local\Temp\InvoiceCheckList.exe**"  
"_The cmdline.cmdline plugin in Volatility 3 is designed to retrieve the command-line arguments that were used to start each process. This can be particularly useful for identifying the full path of an executable and understanding the context in which a process was started._"   

**Q2:**  By determining which executable is utilized by the malware to ensure its persistence, we can strategize for the eradication phase. Which executable is responsible for the malware's persistence?   
**A:**  'schtasks.exe'   

> Based on the process history above, It is likely that the malware is using scheduled tasks for persistence.   

**Q3:** Understanding child processes reveals potential malicious behavior in incidents. Aside from the PowerShell processes, what other active suspicious process, originating from the same parent process, is identified?   
**A:**  'RegSvcs.exe'  

> RegSvcs.exe can be used to load and execute .NET assemblies. Malware can leverage this capability to load malicious .NET assemblies into memory, thereby executing its payload.   

**Q4:** Analyzing malicious process parameters uncovers intentions like defense evasion for hidden, stealthy malware. What PowerShell cmdlet used by the malware for defense evasion?    
**A:**  'Add-MpPreference'

C:\Windows\System32\WindowsPowerShell\v1.0\powershell.exe" Add-MpPreference -ExclusionPath "C:\Users\Lee\AppData\Local\Temp\InvoiceCheckList.exe"

> The Add-MpPreference cmdlet adds an exclusion to Windows Defender. Specifically, it excludes the file InvoiceCheckList.exe located at C:\Users\Lee\AppData\Local\Temp\ from being scanned by Windows Defender. Similarily, the same goes for 'HcdmIYYf.exe'.   

**Q5:** Recognizing detection-evasive executables is crucial for monitoring their harmful and malicious system activities. Which two applications were excluded by the malware from the previously altered application's settings?   
**A:** 'InvoiceCheckList.exe:HcdmIYYf.exe'   

**Q6:** Mapping each technique to MITRE provides clarity and aids effective response during incident analysis. What is the MITRE sub-technique ID the PowerShell commands aim to achieve?   
**A:**  'T1562.001'   

**Q7:** SIDs uniquely identify accounts, reveal type, domain/local status, and correlate malicious activities. What's the Security ID (SID) of the user account the malicious processes are running under?

**A:**  'S-1-5-21-1649652813-3480061347-1948202237-1001'

```
**powershell.exe    S-1-5-21-1649652813-3480061347-1948202237-1001   Lee**
powershell.exe    S-1-5-21-1649652813-3480061347-1948202237-513    Domain Users
powershell.exe    S-1-1-0   Everyone
powershell.exe    S-1-5-114   Local Account (Member of Administrators)
powershell.exe    S-1-5-32-544   Administrators
powershell.exe    S-1-5-4   Interactive
powershell.exe    S-1-2-1   Console Logon (Users who are logged onto the physical console)
powershell.exe    S-1-5-11   Authenticated Users
powershell.exe    S-1-5-15   This Organization
powershell.exe    S-1-5-113   Local Account
powershell.exe    S-1-5-5-0-224031   Logon Session
powershell.exe    S-1-2-0   Local (Users with the ability to log in locally)
powershell.exe    S-1-5-64-10   NTLM Authentication
powershell.exe    S-1-16-12288   High Mandatory Level
```   
Based on the assigned SIDS we can infer that user is 'Lee', because every other SID being groups.


`./vol.py -f file.dmp windows.getsids.GetSIDs --pid 6980` # Powershell  
`./vol.py -f file.dmp windows.getsids.GetSIDs --pid 7656` # Powershell  
`./vol.py -f file.dmp windows.getsids.GetSIDs --pid 3512` # Schtasks  